## Imports, function definitions

Note: DeepMoD does not work for python3.9

In [1]:
# pip install pysindy

In [2]:
# pip install DeePyMoD

In [3]:
# pip install --upgrade tensorflow

In [4]:
# pip install tensorflow-io

In [5]:
results_path = ""

In [6]:
import matplotlib.pyplot as plt

# General imports
import numpy as np
import torch
import pandas as pd

# DeePyMoD imports
from deepymod import DeepMoD
from deepymod.data import Dataset, get_train_test_loader
from deepymod.data.samples import Subsample_random
from deepymod.data.burgers import burgers_delta
from deepymod.model.constraint import LeastSquares
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.sparse_estimators import Threshold
from deepymod.training import train
from deepymod.training.sparsity_scheduler import Periodic, TrainTest, TrainTestPeriodic

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


import pickle
from sklearn.metrics import mean_squared_error
def load_data(X,y):
    return torch.from_numpy(X).float(),torch.from_numpy(y).float()

def uxt_2D_to1D(u,x,t):
    '''
    Function for reshaping u,x and t
    '''
    t_reg=[]
    x_reg=[]
    for i in range(len(t)):
        for j in range(len(x)):
            t_reg.append(t[i])
            x_reg.append(x[j])
    u_reg = u.reshape((u.size, 1))

    return u_reg,x_reg,t_reg

descr = ['',
 'u_{x}',
 'u_{xx}',
 'u_{xxx}',
 'u',
 'uu_{x}',
 'uu_{xx}',
 'uu_{xxx}',
 'u^2',
 'u^2u_{x}',
 'u^2u_{xx}',
 'u^2u_{xxx}']

def return_pde(w, rhs_description=descr, ut = 'u_t'):
    pde = ut + ' = '
    first = True
    for i in range(len(w)):
        if w[i] != 0:
            if not first:
                pde = pde + ' + '
            pde = pde + "(%05f)" % (w[i].real) + rhs_description[i] 
            first = False
    return pde


cuda


In [7]:
def error_metric(true_coeff,pred_coeff):
    wrmse = mean_squared_error(true_coeff,pred_coeff,squared=False)
    return wrmse

## Testing 3.1

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(21,23):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  20
Dataset is using device:  cuda


2023-04-28 11:09:26.444465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 11:09:27.012000: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  9250  MSE: 1.61e-03  Reg: 1.56e-05  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.100554)u_{xx} + (-0.968592)uu_{x}  rmse =  0.009068004206201933
Dataset is using device:  cuda
 19750  MSE: 1.63e-03  Reg: 1.05e-05  L1: 1.41e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.100298)u_{xx} + (-0.991227)uu_{x}  rmse =  0.0025340346572110496
Dataset is using device:  cuda
  5750  MSE: 1.66e-03  Reg: 1.21e-05  L1: 1.41e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097923)u_{xx} + (-0.988093)uu_{x}  rmse =  0.003489222773285245
Dataset is using device:  cuda
 16500  MSE: 1.56e-03  Reg: 1.23e-05  L1: 1.46e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.096440)u_{xx} + (-0.984687)uu_{x}  rmse =  0.004538480829322871
Dataset is using device:  cuda
 19750  MSE: 1.63e-03  Reg: 1.16e-05  L1: 1.48e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099205)u_{xx} + (-0

KeyboardInterrupt: 

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(22,23):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  10
Dataset is using device:  cuda


2023-04-29 01:03:45.232196: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 01:03:45.841220: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 18250  MSE: 3.16e-03  Reg: 1.18e-05  L1: 1.45e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.100213)u_{xx} + (-0.970791)uu_{x}  rmse =  0.008432072683007805
Dataset is using device:  cuda
 19750  MSE: 3.22e-03  Reg: 1.67e-05  L1: 1.76e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.096512)u_{xx} + (-1.136541)uu_{x} + (0.296480)u^2u_{x}  rmse =  0.09423196010064223
Dataset is using device:  cuda
 19750  MSE: 3.28e-03  Reg: 1.43e-05  L1: 2.41e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.021181)u_{x} + (0.095191)u_{xx} + (0.047798)u + (-1.072674)uu_{x} + (-0.137422)u^2 + (0.373700)u^2u_{x} + (-0.066890)u^2u_{xx}  rmse =  0.11939049585802719
Dataset is using device:  cuda
 19750  MSE: 3.12e-03  Reg: 9.31e-06  L1: 1.84e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099970)u_{xx} + (-1.127514)uu_{x} + (0.318566)u^2u_{x}  rmse =  0.09905548777345552
Dataset is using device:  cuda
 19750 

## Testing 3.2

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(23,25):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  8
Dataset is using device:  cuda


2023-04-30 12:06:45.495555: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 12:06:46.147400: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 11000  MSE: 4.05e-03  Reg: 1.70e-05  L1: 2.45e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.043343)u_{x} + (0.076341)u_{xx} + (0.045480)u + (-0.956997)uu_{x} + (0.080749)uu_{xx} + (-0.119291)u^2 + (0.226120)u^2u_{x} + (-0.131639)u^2u_{xx}  rmse =  0.08924010725297614
Dataset is using device:  cuda
 19750  MSE: 4.11e-03  Reg: 1.71e-05  L1: 1.69e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098707)u_{xx} + (-1.099530)uu_{x} + (0.222696)u^2u_{x}  rmse =  0.07041615273489531
Dataset is using device:  cuda
 13000  MSE: 4.01e-03  Reg: 1.04e-05  L1: 1.95e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.044045)u_{x} + (0.090376)u_{xx} + (0.048035)u + (-0.836739)uu_{x} + (-0.142238)u^2 + (-0.074487)u^2u_{xx}  rmse =  0.0687834483794961
Dataset is using device:  cuda
 19750  MSE: 3.93e-03  Reg: 1.96e-05  L1: 1.45e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098683)u_{xx} + (-0.973854)uu_{

## Testing 3.3

In [9]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(25,27):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  5
Dataset is using device:  cuda
 13000  MSE: 6.31e-03  Reg: 2.97e-05  L1: 1.55e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.074436)u_{x} + (0.086334)u_{xx} + (0.018176)u + (-0.789501)uu_{x} + (-0.049400)u^2  rmse =  0.06633755902306164
Dataset is using device:  cuda
 11250  MSE: 6.38e-03  Reg: 2.04e-05  L1: 1.38e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.046462)u_{x} + (0.096377)u_{xx} + (-0.861460)uu_{x}  rmse =  0.042195122162286804
Dataset is using device:  cuda
 13250  MSE: 6.44e-03  Reg: 1.92e-05  L1: 1.93e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099735)u_{xx} + (-1.147731)uu_{x} + (0.350593)u^2u_{x}  rmse =  0.10982560511239674
Dataset is using device:  cuda
  5250  MSE: 6.26e-03  Reg: 2.37e-05  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.046513)u_{x} + (0.095463)u_{xx} + (-0.895652)uu_{x}  rmse =  0.033005686698808566
Dataset is using device:

KeyboardInterrupt: 

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(26,27):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  4
Dataset is using device:  cuda


2023-05-03 10:13:20.886922: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 10:13:21.520566: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 13000  MSE: 7.94e-03  Reg: 2.45e-05  L1: 1.66e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.052845)u_{x} + (0.085566)u_{xx} + (0.035844)u + (-0.867127)uu_{x} + (-0.090664)u^2  rmse =  0.050133974275677604
Dataset is using device:  cuda
 11500  MSE: 8.11e-03  Reg: 2.57e-05  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.088940)u_{xx} + (-0.988663)uu_{x}  rmse =  0.004572122752761711
Dataset is using device:  cuda
 16750  MSE: 8.11e-03  Reg: 2.12e-05  L1: 1.42e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.096238)u_{xx} + (-0.998883)uu_{x}  rmse =  0.0011328776550698207
Dataset is using device:  cuda
 15000  MSE: 8.26e-03  Reg: 2.37e-05  L1: 1.48e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.094591)u_{xx} + (-1.039534)uu_{x}  rmse =  0.011518897528463417
Dataset is using device:  cuda
 17250  MSE: 8.21e-03  Reg: 3.55e-05  L1: 1.46e+00 Algorithm converged. Writing model to 

## Testing 3.4

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(27,29):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  3
Dataset is using device:  cuda


2023-05-05 10:53:52.653849: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 10:53:53.284796: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  8750  MSE: 1.09e-02  Reg: 2.74e-05  L1: 1.44e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.061248)u_{x} + (0.092809)u_{xx} + (-0.831632)uu_{x} + (-0.021379)u^2  rmse =  0.05212798953544569
Dataset is using device:  cuda
 15500  MSE: 1.10e-02  Reg: 3.51e-05  L1: 1.41e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.095820)u_{xx} + (-0.994632)uu_{x}  rmse =  0.0019641191632121073
Dataset is using device:  cuda
  9000  MSE: 1.07e-02  Reg: 3.45e-05  L1: 1.41e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.031672)u_{x} + (0.089282)u_{xx} + (-0.856254)uu_{x}  rmse =  0.042603752078816566
Dataset is using device:  cuda
 19750  MSE: 1.02e-02  Reg: 2.63e-05  L1: 1.70e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099238)u_{xx} + (-1.078439)uu_{x} + (0.222167)u^2u_{x}  rmse =  0.06801439359452298
Dataset is using device:  cuda
 18750  MSE: 1.09e-02  Reg: 3.70e-05  L1: 1.46e+00 Algorithm conv

## Testing 3.5

In [9]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(29,31):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  1
Dataset is using device:  cuda
  6750  MSE: 3.38e-02  Reg: 1.04e-04  L1: 2.90e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.085537)u_{x} + (0.069348)u_{xx} + (0.094731)u + (-0.596863)uu_{x} + (-0.092085)uu_{xx} + (0.027504)uu_{xxx} + (-0.292017)u^2 + (-0.049026)u^2u_{xxx}  rmse =  0.15183952172770174
Dataset is using device:  cuda
  6500  MSE: 3.29e-02  Reg: 9.23e-05  L1: 2.16e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.097564)u_{x} + (0.109853)u_{xx} + (0.013348)u_{xxx} + (-0.447994)uu_{x} + (-0.176710)uu_{xx} + (-0.119256)u^2  rmse =  0.1731936678376101
Dataset is using device:  cuda
  5750  MSE: 3.42e-02  Reg: 8.17e-05  L1: 1.79e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.105072)u_{x} + (0.058687)u_{xx} + (0.054959)u + (-0.698839)uu_{x} + (-0.150091)u^2  rmse =  0.10367930890754479
Dataset is using device:  cuda
  7750  MSE: 3.20e-02  Reg: 1.01e-04  L1: 1.42e+00 Algorithm converged. Writ